In [3]:
import pandas as pd
import numpy as np

# For reproducibility
np.random.seed(42)


In [4]:

df = pd.read_csv('dataset.csv')

print("DataFrame shape:", df.shape)
print("DataFrame columns:", df.columns.tolist())
print(df.head())

# Basic statistics
print("\nBasic statistics:\n", df.describe())


DataFrame shape: (10000, 11)
DataFrame columns: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'y']
         x1        x2          x3        x4        x5        x6        x7  \
0  1.205492  5.823226   98.837539 -1.075852  0.999205  0.911543  3.623558   
1  1.391530  3.611581   98.857197 -5.020318  0.677165  0.999492  3.413112   
2  1.692571 -0.887019  100.901276 -0.595548  0.177550 -0.915495  4.320264   
3  4.289320  1.416843  100.784735 -2.897154 -0.066972 -0.786173  2.093003   
4  0.542420 -1.010095  100.015580 -3.070705  0.088324 -0.242669  0.767942   

         x8        x9        x10  y  
0 -1.720267 -0.346191 -54.708330 -1  
1  4.253865  2.041603 -54.317291  1  
2  0.907834  3.126815 -56.397484 -1  
3  1.336237  2.183829 -56.197728  1  
4 -0.284683 -2.104145 -55.794045  1  

Basic statistics:
                  x1            x2            x3            x4            x5  \
count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
mean       1.591

In [5]:


# We must check how y is encoded. If it's in {0,1}, we might convert to {-1, +1}.
# Let's see the unique values of y:
unique_y = df['y'].unique()
print("Unique values of y:", unique_y)

# Convert y to {-1, +1} if it appears to be in {0,1}.
# This is often a choice for Perceptron / SVM convenience.
# If the dataset is already in {-1, +1}, you can skip this step.
if set(unique_y) == {0,1}:
    df['y'] = df['y'].apply(lambda val: 1 if val == 1 else -1)
    print("Converted y from {0,1} to {-1,+1}")
    
# Now let's separate features and labels
X = df[[f'x{i}' for i in range(1,11)]].values  # shape = (n_samples, 10)
y = df['y'].values                             # shape = (n_samples,)

# We will standardize (mean 0, std 1) the features to help training stability.
# But to avoid data leakage, we will do this transformation AFTER splitting 
# into train/val/test. We'll just define a function now.


Unique values of y: [-1  1]


In [6]:
########################################
# JUPYTER NOTEBOOK CELL 4
# (Train/Validation/Test Split)
########################################

def train_val_test_split(X, y, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, shuffle=True, seed=42):
    """
    Splits the dataset into train, validation, and test sets.
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."
    
    n = X.shape[0]
    indices = np.arange(n)
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
        
    train_end = int(train_ratio * n)
    val_end = int((train_ratio + val_ratio) * n)
    
    X_train, y_train = X[indices[:train_end]], y[indices[:train_end]]
    X_val, y_val     = X[indices[train_end:val_end]], y[indices[train_end:val_end]]
    X_test, y_test   = X[indices[val_end:]], y[indices[val_end:]]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y, 0.7, 0.15, 0.15)

print("Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


Shapes:
Train: (7000, 10) (7000,)
Validation: (1500, 10) (1500,)
Test: (1500, 10) (1500,)


In [7]:
########################################
# JUPYTER NOTEBOOK CELL 5
# (Helper Functions)
########################################

def standard_scale_train(X_train):
    """
    Computes mean and std of X_train, returns scaled X_train, 
    plus the (mean, std) for future use on validation/test sets.
    """
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0, ddof=1)
    std[std == 0] = 1.0  # Avoid division by zero if any feature is constant
    X_train_scaled = (X_train - mean) / std
    return X_train_scaled, mean, std

def standard_scale_apply(X, mean, std):
    """
    Applies the saved mean, std to scale a new dataset X.
    """
    return (X - mean) / std

def accuracy_score(y_true, y_pred):
    """
    Computes 0-1 accuracy, i.e., proportion of matching labels.
    """
    return np.mean(y_true == y_pred)

def sign_function(scores):
    """
    Given an array of scores, return an array of +1 or -1 based on the sign.
    """
    return np.where(scores >= 0, 1, -1)


In [8]:
########################################
# JUPYTER NOTEBOOK CELL 6
# (Linear Models)
########################################

# 1) PERCEPTRON
def perceptron_train(X, y, epochs=10, eta=1.0):
    """
    Trains a linear Perceptron classifier.
    X: shape (n_samples, n_features)
    y: shape (n_samples,) in {-1, +1}
    epochs: number of passes over the dataset
    eta: learning rate
    
    Returns:
        w: weight vector, shape (n_features,)
        b: bias term (or we can incorporate into w by adding a constant feature)
    """
    n_samples, n_features = X.shape
    # Initialize parameters
    w = np.zeros(n_features)
    b = 0.0
    
    for _ in range(epochs):
        for i in range(n_samples):
            # Perceptron update rule
            # If y[i]*(w · X[i] + b) <= 0, then update
            if y[i] * (np.dot(w, X[i]) + b) <= 0:
                w += eta * y[i] * X[i]
                b += eta * y[i]
    return w, b

def perceptron_predict(X, w, b):
    """
    Predict with a trained Perceptron classifier.
    """
    scores = X @ w + b
    return sign_function(scores)


# 2) SVM (Pegasos, Hinge Loss)
def pegasos_svm_train(X, y, lam=1e-4, max_iters=1000, batch_size=1):
    """
    Pegasos algorithm for SVM with hinge loss.
    Implementation for binary classification y in {-1, +1}.
    
    X: (n_samples, n_features)
    y: (n_samples,) in {-1, +1}
    lam: regularization parameter
    max_iters: total number of iterations (batches)
    batch_size: mini-batch size
    
    Returns:
        w: weight vector
        (No explicit bias here for simplicity; 
         we can either incorporate a bias as an extra dimension, 
         or treat data as is. For simplicity, let's skip bias.)
    """
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    t = 0
    
    for iteration in range(1, max_iters+1):
        t += 1
        # pick a mini-batch
        batch_indices = np.random.choice(n_samples, batch_size, replace=False)
        X_batch = X[batch_indices]
        y_batch = y[batch_indices]
        
        # compute step size
        eta_t = 1.0 / (lam * iteration)
        
        # gradient step
        # w <- (1 - eta_t * lambda) w
        w = (1 - eta_t * lam) * w
        
        # For each sample in the batch, check hinge condition
        for i in range(batch_size):
            if y_batch[i] * np.dot(w, X_batch[i]) < 1:
                w += eta_t * y_batch[i] * X_batch[i]
        
    return w

def pegasos_svm_predict(X, w):
    """
    Predict with Pegasos SVM (linear).
    """
    scores = X @ w
    return sign_function(scores)


# 3) Regularized Logistic Classification (Pegasos style)
def pegasos_logistic_train(X, y, lam=1e-4, max_iters=1000, batch_size=1):
    """
    Pegasos-style algorithm for logistic loss:
    Minimize: lam/2 * ||w||^2 + average(log(1 + exp(-y_i * w.x_i)))
    We'll do a stochastic gradient approach analogous to Pegasos.
    
    X: (n_samples, n_features)
    y: (n_samples,) in {-1, +1}
    lam: regularization parameter
    max_iters: total number of iterations (batches)
    batch_size: mini-batch size
    
    Returns:
        w: weight vector
    """
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    
    for t in range(1, max_iters+1):
        # sample a mini-batch
        batch_indices = np.random.choice(n_samples, batch_size, replace=False)
        X_batch = X[batch_indices]
        y_batch = y[batch_indices]
        
        eta_t = 1.0 / (lam * t)  # step size
        
        # gradient of regularization
        w = (1 - eta_t * lam) * w
        
        # logistic loss gradient
        for i in range(batch_size):
            # gradient of log(1 + exp(-y_i * w.dot(x_i))) w.r.t w
            # = - (y_i * x_i) * exp(-y_i*w.dot(x_i)) / (1 + exp(-y_i*w.dot(x_i)))
            exponent = - y_batch[i] * np.dot(w, X_batch[i])
            grad_factor = - (y_batch[i] * X_batch[i]) * (np.exp(exponent) / (1.0 + np.exp(exponent)))
            w -= eta_t * grad_factor
    
    return w

def pegasos_logistic_predict(X, w):
    """
    Predict with logistic classifier using sign of w.x.
    """
    scores = X @ w
    return sign_function(scores)


In [9]:
########################################
# JUPYTER NOTEBOOK CELL 7
# (Polynomial Feature Expansion)
########################################

def polynomial_feature_expansion_degree2(X):
    """
    Given X of shape (n_samples, n_features),
    return a new array with polynomial features of degree 2:
      - all original features
      - all pairwise products x_i * x_j (for i <= j, i.e. squares and cross terms)
      
    We can choose i < j if we do not want the squared terms repeated. 
    But let's do i <= j so we explicitly include squares as well:
       x1^2, x2^2, ...
    The resulting dimension = n_features + n_features*(n_features+1)/2
    """
    n_samples, n_features = X.shape
    
    # number of new features
    # original: n_features
    # pairs i <= j: n_features*(n_features+1)//2
    # total = n_features + n_features*(n_features+1)//2
    poly_size = n_features + (n_features*(n_features+1))//2
    
    X_poly = np.zeros((n_samples, poly_size))
    
    # fill in the first n_features
    X_poly[:, :n_features] = X
    
    # now fill in the cross terms
    idx = n_features
    for i in range(n_features):
        for j in range(i, n_features):
            X_poly[:, idx] = X[:, i] * X[:, j]
            idx += 1
    
    return X_poly


In [10]:
########################################
# JUPYTER NOTEBOOK CELL 8
# (Kernel Methods)
########################################

# We define two kernels: Gaussian (RBF) and Polynomial.

def rbf_kernel(X1, X2, gamma):
    """
    Radial Basis Function (Gaussian) kernel:
      k(x,z) = exp(-gamma * ||x - z||^2)
    X1: shape (n1, d)
    X2: shape (n2, d)
    gamma: float
    
    returns K: shape (n1, n2)
    """
    # We can compute pairwise squared distance in a vectorized manner:
    # ||x - z||^2 = ||x||^2 + ||z||^2 - 2 x.z
    X1_sq = np.sum(X1**2, axis=1).reshape(-1, 1)  # shape (n1, 1)
    X2_sq = np.sum(X2**2, axis=1).reshape(1, -1)  # shape (1, n2)
    dist_sq = X1_sq + X2_sq - 2 * np.dot(X1, X2.T)  # shape (n1, n2)
    
    return np.exp(-gamma * dist_sq)

def polynomial_kernel(X1, X2, degree=2, c=1.0):
    """
    Polynomial kernel:
      k(x,z) = (x.z + c)^degree
    X1: shape (n1, d)
    X2: shape (n2, d)
    degree: int
    c: float
    
    returns K: shape (n1, n2)
    """
    return (X1 @ X2.T + c) ** degree


# 1) Kernelized Perceptron
def kernel_perceptron_train(X, y, kernel_func, epochs=10, **kernel_params):
    """
    Kernelized Perceptron.
    X: shape (n_samples, n_features)
    y: shape (n_samples,) in {-1, +1}
    kernel_func: function that takes (X, X, **kernel_params) -> K
    epochs: number of epochs
    **kernel_params: parameters for the kernel
    
    Returns:
        alpha: shape (n_samples,), the dual coefficients
        X (training data) is also stored so we can compute kernel with new points
    """
    n_samples = X.shape[0]
    # Initialize alpha
    alpha = np.zeros(n_samples)
    
    # Precompute the kernel matrix on the training set
    K = kernel_func(X, X, **kernel_params)  # shape (n_samples, n_samples)
    
    for _ in range(epochs):
        for i in range(n_samples):
            # prediction = sign( sum_j alpha_j * y_j * K[i,j] )
            pred = np.sign(np.sum(alpha * y * K[:, i]))
            if pred == 0: 
                pred = 1  # tie-break
            if pred != y[i]:
                alpha[i] += 1
    
    return alpha

def kernel_perceptron_predict(X_train, y_train, alpha, kernel_func, X_test, **kernel_params):
    """
    Predict function for kernelized perceptron.
    alpha, X_train, y_train define the classifier.
    We compute: sign( sum_i alpha_i * y_i * k(x_i, x_test) ).
    """
    K_test = kernel_func(X_train, X_test, **kernel_params)  # shape (n_train, n_test)
    # shape alpha, y_train => (n_train,)
    # We want sum_i alpha_i * y_i * K_test[i, j]
    # final shape => (n_test,)
    decision = np.dot((alpha * y_train), K_test)
    return sign_function(decision)


# 2) Kernelized Pegasos for SVM
def kernel_pegasos_svm_train(X, y, kernel_func, lam=1e-4, max_iters=1000, **kernel_params):
    """
    Kernelized Pegasos algorithm for SVM.
    We store alpha for each training sample.
    Pseudocode reference from "Pegasos: Primal Estimated sub-GrAdient SOlver for SVM" 
    (the kernelized version).
    
    There's a known small typo in some references:
    - Make sure the update uses the correct indices and normalizations.
    - We'll implement the corrected version here.
    
    Returns:
        alpha: shape (n_samples,)
    """
    n_samples = X.shape[0]
    alpha = np.zeros(n_samples)
    
    # Precompute the kernel matrix
    K = kernel_func(X, X, **kernel_params)  # shape (n_samples, n_samples)
    
    for t in range(1, max_iters+1):
        # sample one index i
        i = np.random.randint(0, n_samples)
        eta_t = 1.0 / (lam * t)
        
        # decision value: sum_j alpha_j * y_j * K[j, i]
        decision_i = np.sum(alpha * y * K[:, i])
        
        # check margin
        if y[i] * decision_i < 1:
            alpha[i] = alpha[i] + 1.0 * eta_t
        
        # projection step: 
        # We must ensure sum_j alpha_j y_j K[j,k] <= 1 / lambda for each k 
        # in the dual form. But a simpler approach is to scale alpha if needed
        # so that the objective doesn't exceed constraints. We'll do a 
        # typical Pegasos projection: 
        norm_factor = np.sqrt(lam) * np.sqrt(np.sum((alpha * y)[:,None] * (alpha * y)[None,:] * K))
        if norm_factor > 1.0:
            alpha = alpha / norm_factor
    
    return alpha

def kernel_pegasos_svm_predict(X_train, y_train, alpha, kernel_func, X_test, **kernel_params):
    """
    Predict function for kernelized Pegasos SVM.
    alpha, X_train, y_train define the classifier.
    We compute sign( sum_i alpha_i * y_i * k(x_i, x_test) ).
    """
    K_test = kernel_func(X_train, X_test, **kernel_params)  # shape (n_train, n_test)
    decision = np.dot((alpha * y_train), K_test)
    return sign_function(decision)


In [11]:
########################################
# JUPYTER NOTEBOOK CELL 9
# (Hyperparameter Tuning - Example)
########################################

# For demonstration, let's do a small hyperparameter search for each model.
# In practice, you might do a more thorough grid search or cross-validation.

# 1) Prepare scaled data for linear models
X_train_scaled, mean_train, std_train = standard_scale_train(X_train)
X_val_scaled = standard_scale_apply(X_val, mean_train, std_train)
X_test_scaled = standard_scale_apply(X_test, mean_train, std_train)

# 2) (Optional) Prepare polynomial expanded data
X_train_poly = polynomial_feature_expansion_degree2(X_train_scaled)
X_val_poly = polynomial_feature_expansion_degree2(X_val_scaled)
X_test_poly = polynomial_feature_expansion_degree2(X_test_scaled)

# We define a small function to evaluate a set of hyperparameters on the val set 
# and pick the best. We'll do this for SVM and logistic as an example.


In [12]:
def tune_linear_svm_pegasos(X_tr, y_tr, X_va, y_va, lam_values=[1e-4, 1e-3, 1e-2], 
                            max_iters=1000, batch_size=1):
    best_acc = -1
    best_lam = None
    best_w = None
    for lam in lam_values:
        w = pegasos_svm_train(X_tr, y_tr, lam=lam, max_iters=max_iters, batch_size=batch_size)
        y_pred_val = pegasos_svm_predict(X_va, w)
        acc = accuracy_score(y_va, y_pred_val)
        if acc > best_acc:
            best_acc = acc
            best_lam = lam
            best_w = w
    return best_w, best_lam, best_acc

def tune_logistic_pegasos(X_tr, y_tr, X_va, y_va, lam_values=[1e-4, 1e-3, 1e-2], 
                            max_iters=1000, batch_size=1):
    best_acc = -1
    best_lam = None
    best_w = None
    for lam in lam_values:
        w = pegasos_logistic_train(X_tr, y_tr, lam=lam, max_iters=max_iters, batch_size=batch_size)
        y_pred_val = pegasos_logistic_predict(X_va, w)
        acc = accuracy_score(y_va, y_pred_val)
        if acc > best_acc:
            best_acc = acc
            best_lam = lam
            best_w = w
    return best_w, best_lam, best_acc

# Example usage with the raw scaled features (linear) or polynomial expanded
w_svm_best, lam_svm_best, acc_svm_best = tune_linear_svm_pegasos(X_train_scaled, y_train, 
                                                                 X_val_scaled, y_val,
                                                                 lam_values=[1e-5, 1e-4, 1e-3],
                                                                 max_iters=5000, 
                                                                 batch_size=10)
print("Best Linear SVM (Pegasos) on val set => lam={}, acc={}".format(lam_svm_best, acc_svm_best))

w_svm_best_poly, lam_svm_best_poly, acc_svm_best_poly = tune_linear_svm_pegasos(X_train_poly, y_train, 
                                                                                X_val_poly, y_val,
                                                                                lam_values=[1e-5, 1e-4, 1e-3],
                                                                                max_iters=5000, 
                                                                                batch_size=10)
print("Best Polynomial-Expanded SVM (Pegasos) on val set => lam={}, acc={}".format(lam_svm_best_poly, acc_svm_best_poly))


Best Linear SVM (Pegasos) on val set => lam=1e-05, acc=0.6886666666666666
Best Polynomial-Expanded SVM (Pegasos) on val set => lam=1e-05, acc=0.9253333333333333


In [13]:
########################################
# JUPYTER NOTEBOOK CELL 10
# (Model Evaluation and Comparison)
########################################

# Let's do a final training with the best hyperparams we found for SVM (as an example).
# Then evaluate on test set.

# We'll demonstrate for the linear SVM with scaled features:
y_pred_test = pegasos_svm_predict(X_test_scaled, w_svm_best)
acc_test = accuracy_score(y_test, y_pred_test)
print("Linear SVM (Pegasos) Test Accuracy (with best lam={}): {:.4f}".format(lam_svm_best, acc_test))

# And for the polynomial expansion version:
y_pred_test_poly = pegasos_svm_predict(X_test_poly, w_svm_best_poly)
acc_test_poly = accuracy_score(y_test, y_pred_test_poly)
print("Polynomial-Expanded SVM (Pegasos) Test Accuracy (with best lam={}): {:.4f}".format(lam_svm_best_poly, acc_test_poly))


# Similarly, one would do for logistic or kernel-based methods after choosing best hyperparams.
#
# For instance, let's do a quick demonstration of kernel methods (without thorough tuning here):
# Kernel Perceptron with RBF kernel
gamma_test = 0.1
epochs_test = 5
alpha_kp = kernel_perceptron_train(X_train_scaled, y_train, rbf_kernel, epochs=epochs_test, gamma=gamma_test)
y_val_kp = kernel_perceptron_predict(X_train_scaled, y_train, alpha_kp, rbf_kernel, X_val_scaled, gamma=gamma_test)
acc_val_kp = accuracy_score(y_val, y_val_kp)
print("\nKernel Perceptron (RBF) with gamma={} val accuracy: {:.4f}".format(gamma_test, acc_val_kp))

# Kernel Pegasos SVM with RBF kernel
alpha_k_peg = kernel_pegasos_svm_train(X_train_scaled, y_train, rbf_kernel, lam=1e-4, max_iters=1000, gamma=gamma_test)
y_val_k_peg = kernel_pegasos_svm_predict(X_train_scaled, y_train, alpha_k_peg, rbf_kernel, X_val_scaled, gamma=gamma_test)
acc_val_k_peg = accuracy_score(y_val, y_val_k_peg)
print("Kernel Pegasos (RBF) with gamma={} val accuracy: {:.4f}".format(gamma_test, acc_val_k_peg))


Linear SVM (Pegasos) Test Accuracy (with best lam=1e-05): 0.6847
Polynomial-Expanded SVM (Pegasos) Test Accuracy (with best lam=1e-05): 0.9193

Kernel Perceptron (RBF) with gamma=0.1 val accuracy: 0.9260
Kernel Pegasos (RBF) with gamma=0.1 val accuracy: 0.8373
